# Import necessary packages

In [184]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [185]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [186]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [187]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [188]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [189]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [190]:
len(sales_intermittent)

1301

In [191]:
len(sales_lumpy)

395

In [192]:
len(sales_erratic)

458

In [193]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [194]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1665,FOODS_1_055_CA_1_validation,FOODS_1_055,FOODS_1,FOODS,CA_1,CA,2,5,2,5,...,0,0,1,2,3,12,5,1,1,6
1669,FOODS_1_059_CA_1_validation,FOODS_1_059,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1678,FOODS_1_068_CA_1_validation,FOODS_1_068,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1692,FOODS_1_082_CA_1_validation,FOODS_1_082,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,4,14,8,6,9,10,10,12,8,7
1712,FOODS_1_103_CA_1_validation,FOODS_1_103,FOODS_1,FOODS,CA_1,CA,7,1,1,2,...,1,0,1,2,1,1,3,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,FOODS_3_777_CA_4_validation,FOODS_3_777,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,4,1,1,0,1,0,0,0,0,0
12169,FOODS_3_801_CA_4_validation,FOODS_3_801,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,3,2,2,0,0,0,1,2,3
12177,FOODS_3_809_CA_4_validation,FOODS_3_809,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,1,1,1,0,7,1,2,4,0
12179,FOODS_3_811_CA_4_validation,FOODS_3_811,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,2,7,4,1,0,0,2,3,5,2


# User-defined functions to calculate Metrics and Croston algorithm

In [195]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [196]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [197]:
sales_pattern = sales_erratic.copy()

In [198]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [201]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [202]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [203]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [204]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [205]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [206]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [207]:
len(sample_list_filtered)

159

# Hyperparameter tuning for Croston

In [208]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [209]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_314_CA_3
Progressing: 0.0 %
Currently Running: FOODS_3_387_CA_3
Progressing: 0.63 %
Currently Running: FOODS_3_629_CA_1
Progressing: 1.26 %
Currently Running: FOODS_1_171_CA_2
Progressing: 1.89 %
Currently Running: FOODS_3_672_CA_4
Progressing: 2.52 %
Currently Running: FOODS_3_462_CA_4
Progressing: 3.14 %
Currently Running: FOODS_3_311_CA_2
Progressing: 3.77 %
Currently Running: FOODS_3_109_CA_1
Progressing: 4.4 %
Currently Running: FOODS_3_787_CA_3
Progressing: 5.03 %
Currently Running: FOODS_3_128_CA_2
Progressing: 5.66 %
Currently Running: FOODS_3_665_CA_2
Progressing: 6.29 %
Currently Running: FOODS_3_182_CA_2
Progressing: 6.92 %
Currently Running: FOODS_1_136_CA_3
Progressing: 7.55 %
Currently Running: FOODS_2_085_CA_3
Progressing: 8.18 %
Currently Running: FOODS_1_032_CA_3
Progressing: 8.81 %
Currently Running: FOODS_3_445_CA_3
Progressing: 9.43 %
Currently Running: FOODS_3_109_CA_4
Progressing: 10.06 %
Currently Running: FOODS_3_224_CA_3
Progressing: 

In [210]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_3_314_CA_3,3.0,6.327064,0.05
1,FOODS_3_314_CA_3,0.0,6.175143,0.05
2,FOODS_3_314_CA_3,1.0,6.175143,0.05
3,FOODS_3_314_CA_3,0.0,5.677433,0.05
4,FOODS_3_314_CA_3,1.0,5.677433,0.05
...,...,...,...,...
22891,FOODS_3_811_CA_4,23.0,6.504490,0.90
22892,FOODS_3_811_CA_4,23.0,11.023042,0.90
22893,FOODS_3_811_CA_4,19.0,20.831288,0.90
22894,FOODS_3_811_CA_4,22.0,19.199286,0.90


# Summay all metrics

In [211]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [212]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.08752734238333536 minutes


In [213]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_019_CA_3,0.05,1.803606,49.0777,54.6987,367.5918,20.612636,29.290882
1,FOODS_1_019_CA_3,0.10,2.232423,60.7462,61.6314,385.3708,25.513410,31.540420
2,FOODS_1_019_CA_3,0.15,2.234669,60.8073,62.2894,369.4923,25.539070,30.587178
3,FOODS_1_019_CA_3,0.20,2.085312,56.7432,60.6020,346.1224,23.832136,28.611789
4,FOODS_1_019_CA_3,0.25,1.913511,52.0683,58.3515,324.5933,21.868696,26.669240
...,...,...,...,...,...,...,...,...
2857,FOODS_3_822_CA_3,0.70,0.862958,17.2665,17.1598,17.2083,7.273500,8.085763
2858,FOODS_3_822_CA_3,0.75,0.876068,17.5288,17.4234,17.4659,7.384000,8.236802
2859,FOODS_3_822_CA_3,0.80,0.887833,17.7642,17.6577,17.6979,7.483167,8.386795
2860,FOODS_3_822_CA_3,0.85,0.905382,18.1153,18.0164,18.0590,7.631074,8.535488


# Check what products has unexpected metrics result

In [214]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
792,FOODS_2_165_CA_3,0.05,inf,inf,200.0,NaN,0.184330,0.184330
793,FOODS_2_165_CA_3,0.10,inf,inf,200.0,NaN,0.297423,0.297423
794,FOODS_2_165_CA_3,0.15,inf,inf,200.0,NaN,0.451824,0.451824
795,FOODS_2_165_CA_3,0.20,inf,inf,200.0,NaN,0.657102,0.657102
796,FOODS_2_165_CA_3,0.25,inf,inf,200.0,NaN,0.917276,0.917276
797,FOODS_2_165_CA_3,0.30,inf,inf,200.0,NaN,1.221874,1.221874
798,FOODS_2_165_CA_3,0.35,inf,inf,200.0,NaN,1.537120,1.537120
799,FOODS_2_165_CA_3,0.40,inf,inf,200.0,NaN,1.808520,1.808520
800,FOODS_2_165_CA_3,0.45,inf,inf,200.0,NaN,1.983232,1.983232
801,FOODS_2_165_CA_3,0.50,inf,inf,200.0,NaN,2.038168,2.038168


In [215]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
792,FOODS_2_165_CA_3,0.05,inf,inf,200.0,NaN,0.184330,0.184330
793,FOODS_2_165_CA_3,0.10,inf,inf,200.0,NaN,0.297423,0.297423
794,FOODS_2_165_CA_3,0.15,inf,inf,200.0,NaN,0.451824,0.451824
795,FOODS_2_165_CA_3,0.20,inf,inf,200.0,NaN,0.657102,0.657102
796,FOODS_2_165_CA_3,0.25,inf,inf,200.0,NaN,0.917276,0.917276
797,FOODS_2_165_CA_3,0.30,inf,inf,200.0,NaN,1.221874,1.221874
798,FOODS_2_165_CA_3,0.35,inf,inf,200.0,NaN,1.537120,1.537120
799,FOODS_2_165_CA_3,0.40,inf,inf,200.0,NaN,1.808520,1.808520
800,FOODS_2_165_CA_3,0.45,inf,inf,200.0,NaN,1.983232,1.983232
801,FOODS_2_165_CA_3,0.50,inf,inf,200.0,NaN,2.038168,2.038168


In [216]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
594,FOODS_2_031_CA_3,0.05,0.613153,100.1067,NaN,98.8808,0.875933,1.531085
595,FOODS_2_031_CA_3,0.10,0.613984,100.2422,NaN,98.8385,0.877119,1.530256
596,FOODS_2_031_CA_3,0.15,0.615046,100.4156,NaN,98.7904,0.878637,1.529298
597,FOODS_2_031_CA_3,0.20,0.616414,100.6390,NaN,98.7354,0.880591,1.528187
598,FOODS_2_031_CA_3,0.25,0.618188,100.9287,NaN,98.6718,0.883126,1.526897
...,...,...,...,...,...,...,...,...
2191,FOODS_3_536_CA_2,0.70,1.457500,97.1667,NaN,96.5397,1.457500,2.443141
2192,FOODS_3_536_CA_2,0.75,1.450817,96.7211,NaN,95.9457,1.450817,2.435732
2193,FOODS_3_536_CA_2,0.80,1.441071,96.0714,NaN,95.0794,1.441071,2.425150
2194,FOODS_3_536_CA_2,0.85,1.425531,95.0354,NaN,93.6980,1.425531,2.408832


In [217]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
792,FOODS_2_165_CA_3,0.05,inf,inf,200.0,NaN,0.184330,0.184330
793,FOODS_2_165_CA_3,0.10,inf,inf,200.0,NaN,0.297423,0.297423
794,FOODS_2_165_CA_3,0.15,inf,inf,200.0,NaN,0.451824,0.451824
795,FOODS_2_165_CA_3,0.20,inf,inf,200.0,NaN,0.657102,0.657102
796,FOODS_2_165_CA_3,0.25,inf,inf,200.0,NaN,0.917276,0.917276
797,FOODS_2_165_CA_3,0.30,inf,inf,200.0,NaN,1.221874,1.221874
798,FOODS_2_165_CA_3,0.35,inf,inf,200.0,NaN,1.537120,1.537120
799,FOODS_2_165_CA_3,0.40,inf,inf,200.0,NaN,1.808520,1.808520
800,FOODS_2_165_CA_3,0.45,inf,inf,200.0,NaN,1.983232,1.983232
801,FOODS_2_165_CA_3,0.50,inf,inf,200.0,NaN,2.038168,2.038168


In [218]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [219]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [220]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [221]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 1.257861635220126 %
Percentage of unexpected values of WMAPE is: 1.257861635220126 %
Percentage of unexpected values of SMAPE is: 3.1446540880503147 %
Percentage of unexpected values of MAPE is: 1.257861635220126 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [222]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [223]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.340048,82.026197,80.662614,96.417422,13.531467,16.309259
0.10,1.183461,75.107131,73.812963,84.874204,12.476465,15.400518
0.15,1.137051,73.867697,71.313091,80.791249,11.895711,14.833308
0.20,1.114794,73.162865,70.105595,78.501230,11.513799,14.511258
0.25,1.104126,73.039988,69.249986,77.104198,11.253346,14.295218
0.30,1.096673,73.213437,68.470959,76.210507,11.081241,14.106631
0.35,1.090148,73.502478,67.643799,75.609946,10.944725,13.938342
0.40,1.085792,73.993805,66.903488,75.439014,10.805869,13.794131
0.45,1.083497,74.656176,66.289967,75.640143,10.684271,13.676742


In [224]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.45
The optimum Alpha based on Mean WMAPE is: 0.25
The optimum Alpha based on Mean SMAPE is: 0.6
The optimum Alpha based on Mean MAPE is: 0.4
The optimum Alpha based on Mean MAE is: 0.65
The optimum Alpha based on Mean RMSE is: 0.7


### Get MEDIAN metrics value of each Alpha

In [225]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.030108,63.8010,65.6598,70.0845,6.502744,7.993354
0.10,0.931837,59.2017,61.7017,65.0455,5.808366,7.048317
0.15,0.871356,58.2872,60.8826,59.8073,5.550224,7.064325
0.20,0.864376,57.6037,59.1034,59.1719,5.822403,6.700187
0.25,0.879838,55.9303,58.3515,56.6135,5.793447,6.738881
0.30,0.854944,54.8818,57.0050,55.2842,5.578152,6.751273
0.35,0.854449,55.1940,55.8745,53.4611,5.400549,6.602023
0.40,0.853411,55.2200,56.2175,53.4238,5.515030,6.530380
0.45,0.856718,54.4299,56.0470,53.0926,5.335907,6.467267


In [226]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.4
The optimum Alpha based on Median of WMAPE is: 0.65
The optimum Alpha based on Median of SMAPE is: 0.35
The optimum Alpha based on Median of MAPE is: 0.55
The optimum Alpha based on Median of MAE is: 0.85
The optimum Alpha based on Median of RMSE is: 0.5


# Run Croston model after decide best paramaters

In [227]:
best_alpha = 0.4

In [228]:
dataframe_weekly

,wm_yr_wk,FOODS_1_003_CA_4,FOODS_1_004_CA_2,FOODS_1_019_CA_3,FOODS_1_022_CA_3,FOODS_1_025_CA_4,FOODS_1_027_CA_3,FOODS_1_030_CA_3,FOODS_1_032_CA_3,FOODS_1_034_CA_2,...,FOODS_3_809_CA_4,FOODS_3_811_CA_2,FOODS_3_811_CA_4,FOODS_3_815_CA_3,FOODS_3_816_CA_3,FOODS_3_816_CA_4,FOODS_3_822_CA_2,FOODS_3_822_CA_3,FOODS_3_825_CA_3,FOODS_3_827_CA_3
0,11101,2.0,15.0,30.0,3.0,5.0,12.0,3.0,28.0,1.0,...,0.0,18.0,19.0,0.0,31.0,7.0,22.0,29.0,0.0,0.0
1,11102,7.0,31.0,9.0,10.0,4.0,23.0,1.0,18.0,0.0,...,9.0,31.0,29.0,0.0,55.0,7.0,14.0,36.0,3.0,0.0
2,11103,6.0,28.0,0.0,11.0,6.0,14.0,3.0,17.0,0.0,...,3.0,25.0,31.0,0.0,117.0,3.0,27.0,50.0,7.0,0.0
3,11104,3.0,37.0,2.0,3.0,4.0,31.0,4.0,20.0,1.0,...,9.0,15.0,19.0,0.0,49.0,10.0,16.0,37.0,3.0,0.0
4,11105,7.0,40.0,0.0,8.0,4.0,24.0,3.0,17.0,2.0,...,3.0,0.0,0.0,1.0,61.0,5.0,17.0,34.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,2.0,39.0,50.0,3.0,6.0,17.0,2.0,23.0,3.0,...,11.0,22.0,19.0,0.0,64.0,14.0,11.0,37.0,19.0,0.0
269,11609,23.0,47.0,3.0,4.0,2.0,18.0,0.0,21.0,2.0,...,6.0,34.0,22.0,0.0,83.0,10.0,14.0,39.0,5.0,0.0
270,11610,2.0,56.0,6.0,4.0,4.0,22.0,1.0,22.0,0.0,...,4.0,32.0,14.0,0.0,22.0,19.0,28.0,52.0,0.0,0.0
271,11611,2.0,31.0,11.0,10.0,5.0,14.0,2.0,43.0,1.0,...,4.0,34.0,15.0,0.0,59.0,3.0,18.0,37.0,3.0,0.0


In [229]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [233]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [234]:
df_result_best_params

,Product,Actual Data,Forecast
0,FOODS_1_055_CA_1,21.0,17.874310
1,FOODS_1_055_CA_1,24.0,19.124564
2,FOODS_1_059_CA_1,0.0,1.195801
3,FOODS_1_059_CA_1,0.0,1.195801
4,FOODS_1_068_CA_1,1.0,1.713628
...,...,...,...
911,FOODS_3_809_CA_4,7.0,5.544150
912,FOODS_3_811_CA_4,15.0,16.703974
913,FOODS_3_811_CA_4,32.0,16.054701
914,FOODS_3_816_CA_4,3.0,13.738725


In [235]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [236]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_003_CA_4,0.667298,84.9288,83.4152,165.8555,4.671085,4.820145
1,FOODS_1_004_CA_2,0.722095,32.4943,30.7517,38.7669,12.997706,15.088108
2,FOODS_1_019_CA_3,1.315558,263.1115,133.7627,101.9447,14.471133,14.833177
3,FOODS_1_022_CA_3,0.623371,124.6742,148.4559,65.2799,6.233709,6.240651
4,FOODS_1_025_CA_4,inf,13.5572,14.6139,13.5572,0.677858,0.698720
...,...,...,...,...,...,...,...
453,FOODS_3_816_CA_4,0.664972,90.9961,89.8944,199.4492,8.644633,8.894656
454,FOODS_3_822_CA_2,0.651514,43.4343,40.2872,60.4166,5.863626,7.188892
455,FOODS_3_822_CA_3,0.624823,18.6729,18.7326,19.0961,8.122702,8.122708
456,FOODS_3_825_CA_3,1.193011,108.4556,72.7631,185.2961,5.965057,7.572776


In [237]:
df_result_final.isnull().sum()

Product     0
MASE       76
WMAPE      76
SMAPE      77
MAPE       95
MAE         0
RMSE        0
dtype: int64

In [241]:
df_result_final.MAPE.mean()

79.97239283746555

In [239]:
df_result_final.to_csv('Croston_Erratic_Metrics.csv')